In [ ]:
import pandas as pd
import numpy as np
file = 'outnpy/lv2415_9DB_lv2414_0DB_20241211_1p5_3p5_dt_1us_S2_off_correction_single_gussain_fit.h5py'
df_1us = pd.read_hdf(file, key='winfo')
df_1us


In [ ]:
import process_data
voltage_map = {'3p5v_1p75v':3.50,'3v_1p5v':3.0,'2p5v_1p25v':2.50,'2v_1v':2.0,'1p5v_750mv':1.50,
               '3p25v_1p625v':3.25, '2p75v_1p375v':2.75,'2p25v_1p125v':2.25,'1p75v_875mv':1.75 }
# file_list = 'runlist/scale_file_list'
file_list = 'runlist/scale_file_list_scaled'
with open(file_list, 'r') as list:
    for line in list: 
        file = line.rstrip('\n')
        voltage = file.split('combine_')[1].split('_1p36v_')[0]
        v = voltage_map.get(voltage)        
        df = pd.read_hdf(file, key='winfo')
        # print(df_1us.Ch0_s2_mu[df_1us.Voltage == v].iloc[0])
        mu = df.Ch0_s2_mu[df.Delta_t == 1].iloc[0] - df_1us.Ch0_s2_mu[df_1us.Voltage == v].iloc[0],
        err = np.sqrt((df.Ch0_s2_sigma[df.Delta_t == 1].iloc[0])**2 + (df_1us.Ch0_s2_sigma[df_1us.Voltage == v ].iloc[0])**2)
        PEns_anode = mu / df.S2_width[df.Delta_t == 1].iloc[0]
        PEns_anode_err = mu / df.S2_width[df.Delta_t == 1].iloc[0] * np.sqrt((err/mu)**2 + (2/df.S2_width[df.Delta_t == 1].iloc[0])**2)
        R2ref= PEns_anode/df.PEns_anode[df.Delta_t == 1000].iloc[0]
        R2ref_err = PEns_anode/df.PEns_anode[df.Delta_t == 1000].iloc[0]*np.sqrt((df.PEns_anode_err[df.Delta_t == 1000].iloc[0]/df.PEns_anode[df.Delta_t == 1000].iloc[0]) **2  + (PEns_anode_err/PEns_anode) **2)
        print('======================')
        df.Ch0_s2_mu[df.Delta_t == 1]= mu
        df.Ch0_s2_sigma[df.Delta_t == 1] = err
        df.PEns_anode[df.Delta_t == 1] = PEns_anode
        df.PEns_anode_err[df.Delta_t == 1] = PEns_anode_err
        df.R2ref[df.Delta_t == 1] = R2ref
        df.R2ref_err[df.Delta_t == 1] = R2ref_err
        print(v, mu, err, PEns_anode, PEns_anode_err, R2ref, R2ref_err)
        
        path = file.replace('.h5py', '_corrected_1us.h5py')
        process_data.write_to_hdf5(df, path)